In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from flair.data import Corpus
from flair.datasets import ColumnCorpus

columns = {0: 'text', 1: '_', 2: '_', 3: 'ner'}

data_folder = './data'

corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='valid.txt')
print(corpus)
tag_type = 'ner'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

/home/kurisu/anaconda3/envs/2080/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kurisu/anaconda3/envs/2080/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kurisu/anaconda3/envs/2080/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kurisu/anaconda3/envs/2080/lib/python3.6/site-

2020-03-25 01:39:51,242 Reading data from data
2020-03-25 01:39:51,244 Train: data/train.txt
2020-03-25 01:39:51,245 Dev: data/valid.txt
2020-03-25 01:39:51,246 Test: data/test.txt
Corpus: 14041 train + 3250 dev + 3453 test sentences
Dictionary with 12 tags: <unk>, O, B-ORG, B-MISC, B-PER, I-PER, B-LOC, I-ORG, I-MISC, I-LOC, <START>, <STOP>


In [2]:
from flair.embeddings import ELMoEmbeddings,BertEmbeddings,FlairEmbeddings
from flair.models import SequenceTagger
from ensemble_tagger import EnsembleTagger
from typing import List

elmo_tagger = SequenceTagger(hidden_size=256,
                             embeddings=ELMoEmbeddings('small'),
                             tag_dictionary=tag_dictionary,
                             tag_type=tag_type,
                             use_crf=True)
bert_tagger = SequenceTagger(hidden_size=256,
                             embeddings=BertEmbeddings(),
                             tag_dictionary=tag_dictionary,
                             tag_type=tag_type,
                             use_crf=True)
flair_tagger = SequenceTagger(hidden_size=256,
                              embeddings=FlairEmbeddings('en-forward'),
                              tag_dictionary=tag_dictionary,
                              tag_type=tag_type,
                              use_crf=True)
ensemble_tagger = EnsembleTagger(models=[elmo_tagger, bert_tagger, flair_tagger],
                                 tag_dictionary=tag_dictionary,
                                 tag_type=tag_type,
                                 use_crf=True,
                                 mode='loss')
print(str(ensemble_tagger))

Ensemble Tagger: [
SequenceTagger(
  (embeddings): ELMoEmbeddings(model=elmo-small)
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=768, out_features=768, bias=True)
  (rnn): LSTM(768, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=12, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
),
SequenceTagger(
  (embeddings): BertEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
          

In [3]:
from flair.trainers import ModelTrainer
from datetime import datetime

trainer: ModelTrainer = ModelTrainer(ensemble_tagger, corpus)
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')

trainer.train("./log/%s_%s/" % ('ensemble', str(timestamp)),
              learning_rate=0.01,
              mini_batch_size=64,
              max_epochs=150)

2020-03-25 01:40:38,818 ----------------------------------------------------------------------------------------------------
2020-03-25 01:40:38,824 Model: "Ensemble Tagger: [
SequenceTagger(
  (embeddings): ELMoEmbeddings(model=elmo-small)
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=768, out_features=768, bias=True)
  (rnn): LSTM(768, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=12, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
),
SequenceTagger(
  (embeddings): BertEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): Be

2020-03-25 01:40:38,825 ----------------------------------------------------------------------------------------------------
2020-03-25 01:40:38,827 Corpus: "Corpus: 14041 train + 3250 dev + 3453 test sentences"
2020-03-25 01:40:38,828 ----------------------------------------------------------------------------------------------------
2020-03-25 01:40:38,830 Parameters:
2020-03-25 01:40:38,830  - learning_rate: "0.01"
2020-03-25 01:40:38,831  - mini_batch_size: "64"
2020-03-25 01:40:38,833  - patience: "3"
2020-03-25 01:40:38,834  - anneal_factor: "0.5"
2020-03-25 01:40:38,835  - max_epochs: "150"
2020-03-25 01:40:38,835  - shuffle: "True"
2020-03-25 01:40:38,836  - train_with_dev: "False"
2020-03-25 01:40:38,837  - batch_growth_annealing: "False"
2020-03-25 01:40:38,837 ----------------------------------------------------------------------------------------------------
2020-03-25 01:40:38,838 Model training base path: "log/ensemble_20200325014038"
2020-03-25 01:40:38,839 -------------

2020-03-25 01:47:10,095 epoch 6 - iter 66/220 - loss 12.90190447 - samples/sec: 240.96
2020-03-25 01:47:16,938 epoch 6 - iter 88/220 - loss 12.81019738 - samples/sec: 215.76
2020-03-25 01:47:23,796 epoch 6 - iter 110/220 - loss 12.64992247 - samples/sec: 215.40
2020-03-25 01:47:30,594 epoch 6 - iter 132/220 - loss 12.59430048 - samples/sec: 216.69
2020-03-25 01:47:37,397 epoch 6 - iter 154/220 - loss 12.60528687 - samples/sec: 217.87
2020-03-25 01:47:44,213 epoch 6 - iter 176/220 - loss 12.63301843 - samples/sec: 216.53
2020-03-25 01:47:51,062 epoch 6 - iter 198/220 - loss 12.62941765 - samples/sec: 217.99
2020-03-25 01:47:58,053 epoch 6 - iter 220/220 - loss 12.60491306 - samples/sec: 211.21
2020-03-25 01:47:58,374 ----------------------------------------------------------------------------------------------------
2020-03-25 01:47:58,375 EPOCH 6 done: loss 12.6049 - lr 0.0050
2020-03-25 01:48:03,990 DEV : loss 13.233296394348145 - score 0.0
2020-03-25 01:48:04,080 BAD EPOCHS (no impro

2020-03-25 01:54:56,522 epoch 12 - iter 132/220 - loss 12.43935035 - samples/sec: 240.58
2020-03-25 01:55:03,409 epoch 12 - iter 154/220 - loss 12.44124749 - samples/sec: 214.22
2020-03-25 01:55:10,244 epoch 12 - iter 176/220 - loss 12.47518829 - samples/sec: 215.52
2020-03-25 01:55:16,491 epoch 12 - iter 198/220 - loss 12.48590815 - samples/sec: 236.98
2020-03-25 01:55:23,253 epoch 12 - iter 220/220 - loss 12.48101177 - samples/sec: 218.85
2020-03-25 01:55:23,570 ----------------------------------------------------------------------------------------------------
2020-03-25 01:55:23,571 EPOCH 12 done: loss 12.4810 - lr 0.0025
2020-03-25 01:55:29,171 DEV : loss 13.035421371459961 - score 0.0
2020-03-25 01:55:29,262 BAD EPOCHS (no improvement): 3
2020-03-25 01:55:29,309 ----------------------------------------------------------------------------------------------------
2020-03-25 01:55:35,888 epoch 13 - iter 22/220 - loss 12.85709125 - samples/sec: 214.22
2020-03-25 01:55:42,847 epoch 13

2020-03-25 02:02:41,419 epoch 18 - iter 176/220 - loss 12.41774207 - samples/sec: 210.69
2020-03-25 02:02:47,652 epoch 18 - iter 198/220 - loss 12.45098725 - samples/sec: 238.49
2020-03-25 02:02:53,629 epoch 18 - iter 220/220 - loss 12.46266841 - samples/sec: 248.71
2020-03-25 02:02:53,944 ----------------------------------------------------------------------------------------------------
2020-03-25 02:02:53,946 EPOCH 18 done: loss 12.4627 - lr 0.0006
2020-03-25 02:02:59,556 DEV : loss 12.99996566772461 - score 0.0
2020-03-25 02:02:59,647 BAD EPOCHS (no improvement): 1
2020-03-25 02:02:59,686 ----------------------------------------------------------------------------------------------------
2020-03-25 02:03:06,128 epoch 19 - iter 22/220 - loss 12.06011144 - samples/sec: 218.75
2020-03-25 02:03:13,087 epoch 19 - iter 44/220 - loss 12.52722391 - samples/sec: 211.92
2020-03-25 02:03:19,997 epoch 19 - iter 66/220 - loss 12.39395517 - samples/sec: 213.72
2020-03-25 02:03:26,925 epoch 19 - 

2020-03-25 02:10:22,187 ----------------------------------------------------------------------------------------------------
2020-03-25 02:10:22,188 EPOCH 24 done: loss 12.4431 - lr 0.0003
2020-03-25 02:10:28,518 DEV : loss 12.987386703491211 - score 0.0
2020-03-25 02:10:28,608 BAD EPOCHS (no improvement): 3
2020-03-25 02:10:28,657 ----------------------------------------------------------------------------------------------------
2020-03-25 02:10:35,133 epoch 25 - iter 22/220 - loss 12.34693722 - samples/sec: 217.61
2020-03-25 02:10:41,952 epoch 25 - iter 44/220 - loss 12.44679553 - samples/sec: 216.63
2020-03-25 02:10:48,874 epoch 25 - iter 66/220 - loss 12.24048805 - samples/sec: 214.91
2020-03-25 02:10:55,843 epoch 25 - iter 88/220 - loss 12.42086354 - samples/sec: 211.86
2020-03-25 02:11:02,860 epoch 25 - iter 110/220 - loss 12.45337475 - samples/sec: 210.23
2020-03-25 02:11:09,969 epoch 25 - iter 132/220 - loss 12.45133888 - samples/sec: 207.33
2020-03-25 02:11:16,841 epoch 25 - 

2020-03-25 02:16:46,188 ----------------------------------------------------------------------------------------------------


{'test_score': 0.0,
 'dev_score_history': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'train_loss_history': [14.466736828197133,
  13.119806853207676,
  12.911857024106112,
  12.769557666778564,
  12.67998833656311,
  12.604913061315363,
  12.571367623589255,
  12.531742598793723,
  12.507466359571977,
  12.480057512630117,
  12.492349529266358,
  12.481011772155762,
  12.490209449421275,
  12.476814046773043,
  12.47074051770297,
  12.465651442787864,
  12.462867199290882,
  12.462668414549395,
  12.454918432235718,
  12.466169734434647,
  12.45063395500183,
  12.446241855621338,
  12.450728984312578,
  12.443070324984463,
  12.454685575311833,
  12.440428621118718,
  12.448289888555353,
  12.453195177425037,
  12.455946124683727],
 'dev_loss_history': [tensor(14.2794, device='cuda:0'),
  tensor(13.9613, device='cuda:0'),
  ten

In [ ]:
sentence = corpus.test[0]

for entity in sentence.get_spans('ner'):
    print(entity)

for token in sentence.tokens:
    print(str(token.get_tag("ner")))
    print(str(token.get_tags_proba_dist("ner")))

ensemble_tagger.predict(sentence,all_tag_prob=True)

for token in sentence.tokens:
    print(token.get_tag("ner").value)
    print(token.get_tags_proba_dist("ner"))

In [ ]:
real = []
for sentence in corpus.test:
    for token in sentence.tokens:
        real.append(token.get_tag("ner").value)
        
elmo_pred = []
bert_pred = []
flair_pred = []
ensemble_pred = []
for sentence in corpus.test:
    ensemble_tagger.predict(sentence,all_tag_prob=True)
    for token in sentence.tokens:
        ensemble_pred.append(token.get_tag("ner").value)
    
for sentence in corpus.test:
    elmo_tagger.predict(sentence,all_tag_prob=True)
    for token in sentence.tokens:
        elmo_pred.append(token.get_tag("ner").value)
    
for sentence in corpus.test:
    bert_tagger.predict(sentence,all_tag_prob=True)
    for token in sentence.tokens:
        bert_pred.append(token.get_tag("ner").value)

for sentence in corpus.test:
    flair_tagger.predict(sentence,all_tag_prob=True)
    for token in sentence.tokens:
        flair_pred.append(token.get_tag("ner").value)
    

In [ ]:
from conlleval import evaluate

print(evaluate(real, ensemble_pred))
print(evaluate(real, elmo_pred))
print(evaluate(real, bert_pred))
print(evaluate(real, flair_pred))